- ### 바이낸스 API를 이용해서 
### `2023년 9월15일~2023년 9월 26일까지의 매분당 비트 코인 가격`을 수집


- ### 수집한 비트코인 가격을 `MySQL`에 저장


In [7]:
import requests
import pandas as pd
from datetime import datetime
import time
import pymysql
from tqdm import tqdm

In [69]:
# DB 연결
db = pymysql.connect(
    host='localhost',
    port=3306,
    user='root',
    passwd='8489',
    db='coin_db',
    charset='utf8'
)

In [70]:
# DB 쿼리를 실행할 객체
cursor = db.cursor()

In [57]:
# 비트코인 정보를 가져올 바이넌스 API 주소
url = "http://api.binance.com/api/v3/klines"

# 비트코인 종류 BTCUSDT => 비트코인
coin_name = "BTCUSDT"

# 수집 시작 날짜
start_date = '2023-09-15'

# 수집 마지막 날짜
end_date = '2023-09-26'

In [12]:
# 문자열 형태의 start_date, 0시 0분을 datetime 으로 변환 (변환 결과는 1초 단위)
datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple()

time.struct_time(tm_year=2023, tm_mon=9, tm_mday=15, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=258, tm_isdst=-1)

In [13]:
# 위의 변환 결과를 정수로 변환
## 이를 1000을 곱해 1/1000 단위로 변환

int(time.mktime(datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000

1694703600000

In [74]:
# end_date는 23시 59분을 기준으로 start_date와 똑같이 하여
## 각각 start , end 변수에 할당

start = int(time.mktime(datetime.strptime(start_date + ' 00:00', '%Y-%m-%d %H:%M').timetuple())) * 1000

end = int(time.mktime(datetime.strptime(end_date + ' 23:59', '%Y-%m-%d %H:%M').timetuple())) * 1000

In [72]:
start

1694703600000

In [75]:
# 바이넌스 API 파라미터 정보
params = {
    'symbol' : coin_name,
    'interval' : '1m',  #수집 시간 간격 1분
    'limit' : 1000,     #한번에 가져올 데이터 수
    'startTime' : start,
    'endTime' : end
}

In [76]:
# 시작 시간부터 끝나는 시간까지

# while문에서 tqdm을 직접 쓰기 어려워
## 따로 pbar을 만들어 tqdm, 진행상황을 표현
pbar = tqdm(total=end - start + 1)

while start < end:
    params['startTime'] = start     #시작시간 갱신
    
    #바이낸스 API로부터 가져온 결과를 저장
    result = requests.get(url,params=params)
    coin_list = result.json()

    if not coin_list:
        break
    
    # 코인 가격 하나씩 i 변수에 저장
    for i in coin_list:
        # print("coin= ", i)
        
        # coin[0] : 비트코인 날짜와 시간이 정수값으로 저장되어있으므로
        ## 1000으로 나눠 1초단위로 변환 후 datetime.fromtimestamp을 통해 datetime 형태로 변환
        open_time = datetime.fromtimestamp(i[0] // 1000) # 몫의 값만 필요하므로 //
        open_price = i[1] # 비트코인의 시가
        high_price = i[2] # 고가
        low_price = i[3] # 저가
        close_price = i[4] # 종가
        volume = i[5] # 거래량
        
        # coin 테이블에 비트코인 정보를 저장할 쿼리
        sql = 'insert into coin (open_time,open_price,high_price,low_price,close_price,volume,symbol) '
        sql += '    values(%s,      %s,     %s,     %s,     %s,     %s,     %s)'
        
        # sql 쿼리 실행 (%s에 변수의 값 대입)
        cursor.execute(sql,(open_time,open_price,high_price,low_price,close_price,volume,coin_name))
        
        # 커밋 실행 (DB 저장)
        db.commit()
        # print(f"{open_time} : get")
        
    # coin_list[-1][0] : 코인 리스트의 마지막 행(-1) 0번째 열 -> 수집한 마지막 시간이 int로 저장되어있음
    ## 단위는 1/1000 초
    ### 60000 -> 60초 후 시간을 시작 시간으로 데이터 수집
    start = coin_list[-1][0] + 60000    #다음 단계, 1분 뒤의 결과
    time.sleep(1)
    
    # tqdm 업데이트
    pbar.update(1)
    
# tqdm 종료
pbar.close()

# while문 종료, DB 연결 종료
print("비트코인 수집 완료")
db.close()

### 데이터 limit 가 1000 으로 제한되어있음.

  0%|          | 17/1036740001 [00:31<541690:49:17,  1.88s/it]

비트코인 수집 완료
